# NanoAOD precision checks

> “Many variables are stored with limited precision (i.e. less than 32bits float) by zeroing a given number of bits in the float mantissa (this results in a higher compression when stored on disk)."

* Get gen kinematic information from MiniAOD and compared with gen information stored in NanoAOD events.
  - Using [/GluGluToRadionToHHTo2B2WToLNu2J_M-250_narrow_13TeV-madgraph/RunIISummer16MiniAODv3-PUMoriond17_94X_mcRun2_asymptotic_v3-v2/MINIAODSIM](https://cmsweb.cern.ch/das/request?instance=prod/global&input=file+dataset%3D%2FGluGluToRadionToHHTo2B2WToLNu2J_M-250_narrow_13TeV-madgraph%2FRunIISummer16MiniAODv3-PUMoriond17_94X_mcRun2_asymptotic_v3-v2%2FMINIAODSIM)
  - Dedicated (very simple) analyzer -->  produces tree with $H \to h(b\bar{b})h(WW^{*})$ gen kinematics.
  - NanoAOD from MiniAOD using "official" configuration using cmsDriver 
  - No "postprocessing" of NanoAOD tuple to avoid any potential errors.

# Gen Particles in MiniAOD

* `prunedGenParticles`
  - only a subset (about 50-100 per event) of the gen particle are stored
  - keep the most important information [prunedGenParticles_cfi.py](https://gitlab.cern.ch/cms-sw/cmssw/-/blob/master/PhysicsTools/PatAlgos/python/slimming/prunedGenParticles_cfi.py)

```python
prunedGenParticles = cms.EDProducer("GenParticlePruner",
    src = cms.InputTag("genParticles"),
    select = cms.vstring(
        "drop  *", # this is the default
        "++keep abs(pdgId) == 11 || abs(pdgId) == 13 || abs(pdgId) == 15",# keep leptons, with history
        "drop   status == 2",                                             # drop the shower part of the history
        "keep abs(pdgId) == 11 || abs(pdgId) == 13 || abs(pdgId) == 15",  # keep leptons (also status1)
        "keep abs(pdgId) == 12 || abs(pdgId) == 14 || abs(pdgId) == 16",  # keep neutrinos
        "+keep pdgId == 22 && status == 1 && (pt > 10 || isPromptFinalState())", # keep gamma above 10 GeV (or all prompt) and its first parent
        "+keep abs(pdgId) == 11 && status == 1 && (pt > 3 || isPromptFinalState())", # keep first parent of electrons above 3 GeV (or prompt)
        "keep++ abs(pdgId) == 15",                # but keep keep taus with decays
	    "drop  status > 30 && status < 70 ", 	  # remove pythia8 garbage
	    "drop  pdgId == 21 && pt < 5",            # remove pythia8 garbage
        "drop   status == 2 && abs(pdgId) == 21", # but remove again gluons in the inheritance chain
        "keep abs(pdgId)==23 || abs(pdgId)==24 || abs(pdgId)==25 || abs(pdgId)==6 || abs(pdgId)==37 ",
  ...
```

# Gen Particles in NanoAOD

configuration fragment: [PhysicsTools.NanoAOD.common_cff](https://gitlab.cern.ch/cms-sw/cmssw/-/blob/master/PhysicsTools/NanoAOD/python/genparticles_cff.py)

* Built from `prunedGenParticles`
  - Additional filters
  - limited precision nbits = 8
  
```python
##################### Tables for final output and docs ##########################
genParticleTable = cms.EDProducer("SimpleCandidateFlatTableProducer",
    src = cms.InputTag("finalGenParticles"),
    cut = cms.string(""), #we should not filter after pruning
    name= cms.string("GenPart"),
    doc = cms.string("interesting gen particles "),
    singleton = cms.bool(False), # the number of entries is variable
    extension = cms.bool(False), # this is the main table for the taus
    variables = cms.PSet(
         pt  = Var("pt",  float, precision=8),
         phi = Var("phi", float,precision=8),
         eta  = Var("eta",  float,precision=8),
    ...
    )
)
```

# MiniAOD Analyzer

* Saves kinematics of $h^{0}$, $b$ quarks, and $W$ bosons.

```c
   // Pruned collection contains most important information of the event
   Handle<edm::View<reco::GenParticle> > pruned;
   iEvent.getByToken(prunedGenToken_, pruned);

   // Packed partcles are all status 1.
   Handle<edm::View<pat::PackedGenParticle> > packed;
   iEvent.getByToken(packedGenToken_, packed);

   std::vector<const Candidate*> higgses;
   std::vector<const Candidate*> bquarks;
   std::vector<const Candidate*> wbosons;

   for(size_t i=0; i<pruned->size(); ++i){
      const Candidate * gencand = &(*pruned)[i];
      if(gencand->pdgId()==25 && gencand->mother(0)->pdgId()==35)
         higgses.push_back(gencand);
      else if(abs(gencand->pdgId())==5 && gencand->mother(0)->pdgId()==25)
         bquarks.push_back(gencand);
      else if(abs(gencand->pdgId())==24 && gencand->mother(0)->pdgId()==25)
         wbosons.push_back(gencand);
      else
         continue;
         //std::cout << gencand->pdgId() << " " << gencand->status() << " " << gencand->mother(0)->pdgId() <<std::endl;
   }
```

In [9]:
import os
import uproot
import numpy as np
import pandas as pd
import awkward as ak
import numba as nb

In [10]:
nanotree = uproot.open("CMSSW_10_2_26/src/HhhAnalysis/GenAnalyzer/test/radion_250_NANO.root:Events")
nanotree1 = uproot.open("CMSSW_10_2_26/src/HhhAnalysis/GenAnalyzer/test/radion_250_NANO-1.root:Events")
mytree = uproot.open("CMSSW_10_2_26/src/HhhAnalysis/GenAnalyzer/test/genBBWW.root:genbbww/Events")

# Filtering NanoAOD Gen Particles

* Only Gen branches

In [5]:
nano = nanotree.arrays(filter_name="GenPart*")
nano.fields

['GenPart_eta',
 'GenPart_mass',
 'GenPart_phi',
 'GenPart_pt',
 'GenPart_genPartIdxMother',
 'GenPart_pdgId',
 'GenPart_status',
 'GenPart_statusFlags']

* Find b quarks from $h^0$

In [12]:
cut_b = (np.abs(nano.GenPart_pdgId) == 5)
mother_b = nano.GenPart_genPartIdxMother[cut_b]
cut_mother_b = (nano.GenPart_pdgId[mother_b] == 25)

In [16]:
b_only = nano[cut_b][cut_mother_b]
b_only.GenPart_pdgId

<Array [[5, -5], [5, -5], ... [5, -5], [5, -5]] type='38740 * var * int32'>

In [25]:
mydict = {
    "event": nanotree.arrays("event", library="np")["event"],
    "b1_pt": ak.to_numpy(b_only.GenPart_pt[:,0]),
    "b1_eta": ak.to_numpy(b_only.GenPart_eta[:,0]),
    "b1_phi": ak.to_numpy(b_only.GenPart_phi[:,0]),
    "b2_pt": ak.to_numpy(b_only.GenPart_pt[:,1]),
    "b2_eta": ak.to_numpy(b_only.GenPart_eta[:,1]),
    "b2_phi": ak.to_numpy(b_only.GenPart_phi[:,1])
}

In [26]:
df =  pd.DataFrame.from_dict(mydict)

In [28]:
branches = ["event", "b1_pt", "b1_eta", "b1_phi", "b2_pt", "b2_eta", "b2_phi"]
genpart_mini = mytree.arrays(branches, library="pd")

# Compare event by event

## NanoAOD

In [40]:
df.sort_values(by=['event']).head(5)

,event,b1_pt,b1_eta,b1_phi,b2_pt,b2_eta,b2_phi
13,67349,59.125,1.574219,2.375000,124.750,1.390625,0.769531
1,67350,53.375,1.347656,1.300781,63.250,0.578125,-1.964844
5,67351,67.750,-0.475586,1.687500,57.625,-0.605469,-1.273438
6,67352,58.625,1.238281,-1.023438,77.000,0.773438,3.039062
4,67353,56.500,-1.359375,-0.560547,62.875,-0.757812,2.578125


## MiniAOD

In [41]:
genpart_mini.sort_values(by=['event']).head(5)

,event,b1_pt,b1_eta,b1_phi,b2_pt,b2_eta,b2_phi
9,67349,59.162838,1.574972,2.373825,124.764183,1.388719,0.768793
0,67350,53.312710,1.348135,1.301964,63.249348,0.579030,-1.965756
4,67351,67.696457,-0.475685,1.686518,57.592220,-0.605304,-1.273538
6,67352,58.662441,1.239945,-1.022804,77.032257,0.774000,3.035913
5,67353,56.497955,-1.359893,-0.560886,62.832787,-0.756951,2.579374


# Change precision

* Produce a new NanoAOD tuple changing precison in configuration.

* Set precision to 23 (no loss of precision)

```python
process.genParticleTable.variables = cms.PSet(
    pt  = Var("pt", float, precision=23),
    phi = Var("phi", float, precision=23),
    eta  = Var("eta", float, precision=23),
    ...
```

In [33]:
nano1 = nanotree1.arrays(filter_name="GenPart*")

In [34]:
cut_b_1 = (np.abs(nano1.GenPart_pdgId) == 5)
mother_b_1 = nano1.GenPart_genPartIdxMother[cut_b_1]
cut_mother_b_1 = (nano1.GenPart_pdgId[mother_b_1] == 25)

In [35]:
b_only_1 = nano1[cut_b_1][cut_mother_b_1]
b_only_1.GenPart_pdgId

<Array [[5, -5], [5, -5], ... [5, -5], [5, -5]] type='38740 * var * int32'>

In [37]:
mydict1 = {
    "event": nanotree1.arrays("event", library="np")["event"],
    "b1_pt": ak.to_numpy(b_only_1.GenPart_pt[:,0]),
    "b1_eta": ak.to_numpy(b_only_1.GenPart_eta[:,0]),
    "b1_phi": ak.to_numpy(b_only_1.GenPart_phi[:,0]),
    "b2_pt": ak.to_numpy(b_only_1.GenPart_pt[:,1]),
    "b2_eta": ak.to_numpy(b_only_1.GenPart_eta[:,1]),
    "b2_phi": ak.to_numpy(b_only_1.GenPart_phi[:,1])
}

# Compare again

## NanoAOD (no loss of precision)

In [42]:
df1 =  pd.DataFrame.from_dict(mydict1)

In [43]:
df1.sort_values(by=['event']).head(5)

,event,b1_pt,b1_eta,b1_phi,b2_pt,b2_eta,b2_phi
16,67349,59.162838,1.574972,2.373825,124.764183,1.388719,0.768793
2,67350,53.312710,1.348135,1.301964,63.249348,0.579030,-1.965756
6,67351,67.696457,-0.475685,1.686518,57.592220,-0.605304,-1.273539
5,67352,58.662441,1.239945,-1.022804,77.032257,0.774000,3.035913
4,67353,56.497955,-1.359894,-0.560886,62.832787,-0.756951,2.579374


## MiniAOD

In [44]:
genpart_mini.sort_values(by=['event']).head(5)

,event,b1_pt,b1_eta,b1_phi,b2_pt,b2_eta,b2_phi
9,67349,59.162838,1.574972,2.373825,124.764183,1.388719,0.768793
0,67350,53.312710,1.348135,1.301964,63.249348,0.579030,-1.965756
4,67351,67.696457,-0.475685,1.686518,57.592220,-0.605304,-1.273538
6,67352,58.662441,1.239945,-1.022804,77.032257,0.774000,3.035913
5,67353,56.497955,-1.359893,-0.560886,62.832787,-0.756951,2.579374
